# __Word2Vec-Analyse auf die Gesamtdaten__

Hier wird die Word2Vec Analyse durchgeführt

Funktioniert nur mit einem grossen Datensatz. Mit den Beispiel-Stellenanzeigen ist eine Word2Vec-Auswertung nicht möglich

In [ ]:
# Import benötigter Libraries
import pandas as pd
from gensim.models.phrases import Phrases, Phraser
from time import time 
from collections import defaultdict 
import multiprocessing
from gensim.models import Word2Vec
pd.set_option('display.max_colwidth', 50)
import os

# Working-Directory
os.chdir('U:/Projekte/Stellenmarkt_Analyse/Daten')

# Nur fürs debuggen
# import logging 
# logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
# Einlesen der gesäuberten Gesamtdaten
data = pd.read_csv("Bereinigte_Gesamtdaten.csv", sep=",")
data = data[~data['Text_Anforderungen'].isnull()]

In [ ]:
# Jahr auswählen, für das die Word2Vec-Analyse erstellt werden soll
df = data.loc[data.year == 2021]

In [ ]:
# Auszuwertende Anforderungen in ein neues DataFrame 
df_clean = pd.DataFrame()
df_clean['clean'] = df['Text_Anforderungen']

In [ ]:
# Wörter splitten in den einzelnen Spalten
sent = [row.split() for row in df_clean['clean']]

In [ ]:
# Gensim Phrasen-Modell initiieren. Erstellt Phrasen welche mehr als 15x vorkommen
phrases = Phrases(sent, min_count=15, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]

In [ ]:
# Wird nicht benötigt
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

In [ ]:
# Parameter setzen
w2v_model = Word2Vec(min_count=15, #Mind. 15x muss das Wort vorkommen. Mit Beispieldaten nicht möglich
                     window=5, #5 wörter vor und nach werden berücksichtigt
                     size=300,
                     workers=multiprocessing.cpu_count()-1)

In [ ]:
# Vokabular aufbauen und Modell initialisierung
t = time()
w2v_model.build_vocab(sentences, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
# Modell trainieren
t = time()
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
# Wenn wir das Modell nicht weitertrainieren wollen, wird init_sims aufgerufen. Damit das Modell effizienter wird.
w2v_model.init_sims(replace=True)

In [ ]:
# Tabelle 7 in Bachelorarbeit
# Top 20 kontextähnlichsten Wörter zu cloud finden
w2v_model.wv.most_similar(positive=["cloud"], topn=20)

In [ ]:
# Tabelle 5 in Bachelorarbeit
# Top 20 kontextähnlichsten Wörter zu sap finden
w2v_model.wv.most_similar(positive=["sap"], topn=20)

In [ ]:
w2v_model.wv.most_similar(positive=["englisch"])